In [ ]:
import sys
!conda install --yes cartopy dask netCDF4 bottleneck

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import cartopy as ccrs
from matplotlib import pyplot as plt
%matplotlib inline

Looking at 1979-2018 Rainfall on the state of California, (NOA extRealtime precipitation) as well as the ENSO anomalies

In [ ]:
precipurlbymonth="http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/.CPC/.UNIFIED_PRCP/.GAUGE_BASED/.GLOBAL/.v1p0/.extREALTIME/.rain/T/monthlyAverage/T/%28Jan%201979%29/%28Dec%202018%29/RANGE/X/%28130W%29/%28100W%29/RANGE/Y/25/50/RANGE/dods"
precipds=xr.open_dataset(precipurlbymonth,decode_times=False)
precipds
ninourl="http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/Y/-5/5/RANGEEDGES/X/190/240/RANGEEDGES/%5BX/Y%5Daverage/yearly-anomalies/T/%28Jan%201979%29/%28Dec%202018%29/RANGE/dods "
ninods=xr.open_dataset(ninourl,decode_times=False)
ninods


In [ ]:
plt.figure(figsize=(8, 10))
plt.subplot(211)
precipds.rain.mean(dim='T').plot()

Make a function that separates by month for funky time units 

In [ ]:
def monthfun(month):
    startvalue=month+228.5
    time=np.empty((40))
    time[0]=startvalue
    for i in range(39):
        time[i+1]=time[i]+12
    return time

Function that calculates correlation and formats data for the map

In [ ]:
def pd_corr(x,y,month):
    print(y.shape)
    print(x.shape)
    corrarray=np.empty((52,62))
    rain=precipds.rain.sel(T=monthfun(month))
    sst=ninods.sst
    for i in range(62):
        for j in range(52):
            timeseries1=xr.DataArray(rain.sel(Y=y[j,i],X=x[j,i]))
            timeseries2=xr.DataArray(sst.sel(zlev=0))
            corrarray[j,i]=timeseries1.to_series().corr(timeseries2.to_series())
    return corrarray
def sample_data(month,shape=(62, 52)):
    """Return ``lons``, ``lats`` and ``data`` of some fake data."""
    nlons, nlats = shape
    lats = np.linspace(24.75,50.25, nlats)
    lons= np.linspace(-130.25,-99.75,nlons)
    lons, lats = np.meshgrid(lons, lats)
    data = pd_corr(lons,lats,month)
    return lons, lats, data
correlatejan=sample_data(0)
#correlate[2].shape

Function that plots map

In [ ]:

import cartopy.io.shapereader as shpreader

def main(month,title):
    central_lon, central_lat = -115, 40
    extent = [-130,-100,25, 50]
    ax = plt.axes(projection=ccrs.crs.Robinson())
    ax.set_title(title)
    ax.set_extent(extent)
    ax.gridlines()
    ax.coastlines(resolution='50m')
    lons, lats, data = sample_data(month)
    cb=ax.contourf(lons, lats, data,transform=ccrs.crs.PlateCarree(),cmap='nipy_spectral')
    plt.colorbar(cb, cmap='nipy_spectral', orientation='vertical',ticklocation='auto')
    
    plt.show()

if __name__ == '__main__':
    main(0,"January precip-niño")

In [ ]:
if __name__ == '__main__':
    main(1,"February precip-niño")

In [ ]:
if __name__ == '__main__':
    main(11,"December precip-niño")

In [ ]:
if __name__ == '__main__':
    main(10,"November precip-niño")

Statistical Signifigance: 